<a href="https://colab.research.google.com/github/FeelOneE/price_prediction/blob/main/coin_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/스터디/코인 트레이드/data


/content/drive/MyDrive/스터디/코인 트레이드/data


In [3]:
%ls -al

total 3758
-rw------- 1 root root 3847495 Dec 18 08:07 XRP.csv


In [9]:
import pandas as pd

data = pd.read_csv("XRP.csv")
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,2021-12-18T00:05:00,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,2021-12-18T00:00:00,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,2021-12-17T23:55:00,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,2021-12-17T23:50:00,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,2021-12-17T23:45:00,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [10]:
# 시간 T를 기준으로 
data['candle_date_time_kst'] = data.candle_date_time_kst.str.split('T', expand=True).apply(lambda x : x[1], axis=1)
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,00:05:00,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,00:00:00,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,23:55:00,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,23:50:00,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,23:45:00,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5


In [11]:
data['candle_date_time_kst'] = data.candle_date_time_kst.str.split(':', expand=True).apply(lambda x : (int(x[0])*60)+(int(x[1])) , axis=1)
data.head(5)

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,unit
0,KRW-XRP,2021-12-17T15:05:00,5,986.0,987.0,984.0,987.0,1639753799934,1.785798e+09,1.812305e+06,5
1,KRW-XRP,2021-12-17T15:00:00,0,989.0,989.0,981.0,986.0,1639753500090,2.128017e+09,2.161762e+06,5
2,KRW-XRP,2021-12-17T14:55:00,1435,984.0,989.0,983.0,989.0,1639753198185,2.275873e+09,2.311628e+06,5
3,KRW-XRP,2021-12-17T14:50:00,1430,987.0,990.0,984.0,985.0,1639752896408,3.238981e+09,3.281353e+06,5
4,KRW-XRP,2021-12-17T14:45:00,1425,983.0,987.0,981.0,987.0,1639752599653,5.052325e+09,5.141900e+06,5
